# Advanced table access for ASCII and VO Tables

In the previous tutorial, we saw how the Table class can be used to read in, write out, and manipulate a variety of table types. For many use cases, this will be sufficient, but in some cases, you may need to access certain file formats at a lower level that is more format-specific. We already saw how to access FITS tables with astropy.io.fits in the FITS tutorial, and here we take a look at the [astropy.io.ascii](https://docs.astropy.org/en/stable/io/ascii/) and [astropy.io.votable](https://docs.astropy.org/en/stable/io/votable/) sub-packages for accessing ASCII tables and VO Tables respectively.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Read ASCII tables with astropy.io.ascii</li>
<li>Read VO tables with astropy.io.votable</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy.io.ascii and astropy.io.votable. For more information about the features presented below as well as other available features, you can read the
[astropy.io.ascii](https://docs.astropy.org/en/stable/io/ascii/) and the [astropy.io.votable](https://docs.astropy.org/en/stable/io/votable/) documentation.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

# Accessing ASCII tables

The astropy.io.ascii sub-package contains an advanced framework for reading and writing out ASCII tables, and is what is used behind the scenes when reading and writing ASCII tables with the Table class.

The main functions are [read](http://docs.astropy.org/en/stable/api/astropy.io.ascii.read.html#astropy.io.ascii.read) and [write](http://docs.astropy.org/en/stable/api/astropy.io.ascii.write.html#astropy.io.ascii.write) which can be imported as:

In [2]:
from astropy.io.ascii import read, write

although for readability we recommend importing:

In [3]:
from astropy.io import ascii

and using ``ascii.read`` and ``ascii.write``.

Let's take a look at the following example of ASCII table:

In [4]:
lines = """
objID                   & osrcid            & xsrcid       
----------------------- & ----------------- & -------------
              277955213 & S000.7044P00.7513 & XS04861B6_005
              889974380 & S002.9051P14.7003 & XS03957B7_004
""".lstrip()

Unlike the Table class, the ``read`` function can be given a string containing the table instead of just a filename (which it also supports). This can come in handy if you want to read/write tables to memory. However, ``read`` is not able to guess the format of the above table correctly:

In [5]:
ascii.read(lines)

objID,&,osrcid,&_1,xsrcid
str23,str1,str17,str1,str13
-----------------------,&,-----------------,&,-------------
277955213,&,S000.7044P00.7513,&,XS04861B6_005
889974380,&,S002.9051P14.7003,&,XS03957B7_004


We therefore need to specify details about what kind of table it is, and what the main delimiter is. Is this case, these options are:

In [6]:
ascii.read(lines, format='fixed_width_two_line', delimiter='&')

objID,osrcid,xsrcid
int32,str17,str13
277955213,S000.7044P00.7513,XS04861B6_005
889974380,S002.9051P14.7003,XS03957B7_004


A list of supported files can be found [in the documentation](http://docs.astropy.org/en/stable/io/ascii/index.html#supported-formats) and it is also possible to [define your own formats](http://docs.astropy.org/en/stable/io/ascii/read.html#advanced-customization).

Because this framework allows a lot of control over how the data is parsed, it is even possible to read data from a table such as:

In [7]:
lines2 = """
# This is the start of my data file

Automatically generated by my_script.py at 2012-01-01T12:13:14
Run parameters: None
Column header line:

x:y:z

Data values section:

1:2:3
4:5:6

Run completed at 2012:01-01T12:14:01
""".lstrip()

In this case, we would need to specify the following options:

In [8]:
tab = ascii.read(lines2, header_start=3, data_start=5, data_end=7, delimiter=':')

In [9]:
tab

x,y,z
int32,int32,int32
1,2,3
4,5,6


In [17]:
from astropy.table import Table

In [19]:
Table.read.help('fits')

Table.read(format='fits') documentation

Read a Table object from an FITS file

If the ``astropy_native`` argument is ``True``, then input FITS columns
which are representations of an astropy core object will be converted to
that class and stored in the ``Table`` as "mixin columns".  Currently this
is limited to FITS columns which adhere to the FITS Time standard, in which
case they will be converted to a `~astropy.time.Time` column in the output
table.

Parameters
----------
input : str or file-like object or compatible `astropy.io.fits` HDU object
    If a string, the filename to read the table from. If a file object, or
    a compatible HDU object, the object to extract the table from. The
    following `astropy.io.fits` HDU objects can be used as input:
    - :class:`~astropy.io.fits.hdu.table.TableHDU`
    - :class:`~astropy.io.fits.hdu.table.BinTableHDU`
    - :class:`~astropy.io.fits.hdu.table.GroupsHDU`
    - :class:`~astropy.io.fits.hdu.hdulist.HDUList`
hdu : int or str, optio

There are many more options and formats available, so if you find yourself needing to read in complicated ASCII formats, be sure to check the [astropy.io.ascii documentation](docs.astropy.org/en/stable/io/ascii/index.html)!

## Accessing VO tables

As for ASCII tables, you should be able to read/write VO tables from the Table class, but sometimes you will need more control to access VO-specific features.

To demonstrate how to access VO tables directly, we will use the [data/xmm-log.vot](data/xmm-log.vot) file which contains a subset of rows from the XMM-Newton observation log which was [obtained from VizieR](http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=B/xmm).

In [10]:
from astropy.io.votable import parse

In [11]:
votable = parse('data/xmm-log.vot')

A VO table file consists of resource elements, each of which can contain one or more table. We can loop over these to check what exists:

In [12]:
votable

<VOTABLE>... 1 tables ...</VOTABLE>

In [13]:
for resource in votable.resources:
    for vtable in resource.tables:
        print(type(vtable))

<class 'astropy.io.votable.tree.Table'>


Note that the table returned is **not** a regular astropy Table, it is a different kind of object (even though the class is called the same). If you want to get a regular table out you could do:

In [14]:
vtable.to_table()

Obsno,PropDate,Object,RAJ2000,DEJ2000,Obs0,ObsDur,Image,PPSp,FITS,XSAlink,SASVersion
,s,,deg,deg,s,s,,,,,
str10,str10,str30,float64,float64,str19,int32,str1,str4,str4,str1,str5
0000110101,2002-09-29,XTE J0421+560,64.92542,55.99944,2001-08-19T07:05:23,32913,Y,PPSp,FITS,Y,9.0
0001730101,2004-12-31,,--,--,2002-03-18T06:40:01,25296,N,PPSp,FITS,N,NOPPS
0001730201,2002-05-25,HD159176,263.67495,-32.58167,2001-03-09T12:44:21,17083,Y,PPSp,FITS,Y,9.0
0001730301,2002-05-25,HD159176,263.67495,-32.58167,2001-03-09T17:30:16,9362,N,PPSp,FITS,Y,9.0
0001730401,2002-05-25,HD159176,263.67495,-32.58167,2001-03-09T09:41:25,10859,N,PPSp,FITS,Y,9.0
0001730501,2004-12-31,HD47129,99.34999,6.13528,2002-09-17T18:35:28,21939,N,PPSp,FITS,Y,9.0
0001730601,2004-12-31,HD47129,99.34999,6.13528,2003-03-16T16:01:51,21863,Y,PPSp,FITS,Y,9.0
0001930101,2002-09-18,IRAS F00235+1024,6.52917,10.68917,2001-01-10T18:47:04,26609,Y,PPSp,FITS,Y,9.0


In [15]:
vtable

<Table length=100>
  Obsno     PropDate       Object       RAJ2000  ... FITS XSAlink SASVersion
               s                          deg    ...                        
  str10      str10         str30        float64  ... str4   str1     str5   
---------- ---------- ---------------- --------- ... ---- ------- ----------
0000110101 2002-09-29    XTE J0421+560  64.92542 ... FITS       Y        9.0
0001730101 2004-12-31                         -- ... FITS       N      NOPPS
0001730201 2002-05-25         HD159176 263.67495 ... FITS       Y        9.0
0001730301 2002-05-25         HD159176 263.67495 ... FITS       Y        9.0
0001730401 2002-05-25         HD159176 263.67495 ... FITS       Y        9.0
0001730501 2004-12-31          HD47129  99.34999 ... FITS       Y        9.0
0001730601 2004-12-31          HD47129  99.34999 ... FITS       Y        9.0
0001930101 2002-09-18 IRAS F00235+1024   6.52917 ... FITS       Y        9.0
0001930301 2003-01-16 IRAS F12514+1027 193.50000 ... FITS

However here we are interested in using the 'native' VO table object, which is a little different:

In [16]:
vtable.fields

[<FIELD ID="Obsno" arraysize="10" datatype="char" name="Obsno" ucd="meta.id;obs"/>,
 <FIELD ID="PropDate" arraysize="10" datatype="char" name="PropDate" ucd="meta.note" unit="s"/>,
 <FIELD ID="Object" arraysize="30" datatype="char" name="Object" ucd="meta.id"/>,
 <FIELD ID="RAJ2000" datatype="double" name="RAJ2000" precision="5" ref="J2000" ucd="pos.eq.ra;meta.main" unit="deg" width="9"/>,
 <FIELD ID="DEJ2000" datatype="double" name="DEJ2000" precision="5" ref="J2000" ucd="pos.eq.dec;meta.main" unit="deg" width="9"/>,
 <FIELD ID="Obs0" arraysize="19" datatype="char" name="Obs0" ucd="time.epoch" unit="s"/>,
 <FIELD ID="ObsDur" datatype="int" name="ObsDur" ucd="time.expo" unit="s" width="7"/>,
 <FIELD ID="Image" arraysize="1" datatype="char" name="Image" ucd="meta.ref.url"/>,
 <FIELD ID="PPSp" arraysize="4" datatype="char" name="PPSp" ucd="meta.ref.url"/>,
 <FIELD ID="FITS" arraysize="4" datatype="char" name="FITS" ucd="meta.ref.url"/>,
 <FIELD ID="XSAlink" arraysize="1" datatype="char" 

This interface gives full access to the information stored in the XML, and also allows standard-compliant VO tables to be written back out.

For more information about reading, manipulating, and writing VO tables, we recommend you take a look at the [astropy.io.votable](http://docs.astropy.org/en/stable/io/votable/index.html) documentation.

<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)